# 2. Etapa de preprocesado de texto

Voy a cargar el df que utilicé en la primer parte de la práctica 01_Descarga-y-exploración.

In [ ]:
import pandas as pd

In [ ]:
# Conectar con el almacenamiento de GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# cargar df del notebook anterior
sample_beauty2 = pd.read_csv('/content/drive/MyDrive/sample_beauty')

# vista del df
sample_beauty2.head()

,Unnamed: 0,overall,reviewText
0,0,0,The color that comes out of this sunless tanne...
1,1,0,"I love this brand, but this one didn't work as..."
2,2,0,Smells ok. Not too strong.
3,3,0,I love and cherish my L'Occitane Shea Butter H...
4,4,0,The first time I used this was a telling momen...


Verifico los valores que hay en la columna overall, en el ejercicio anterior se colocaron las puntuaciones 1,2,3 como 0 y 4,5 como 1

In [ ]:
sample_beauty2['overall'].value_counts(ascending=True)

overall
0    7151
1    9981
Name: count, dtype: int64

voy a eliminar la columna que está demás, para quedarme solo con el texto y las calificaciones ya divididas en positivas (1) y negativas (0)

In [ ]:
# Eliminar la columna 'Unnamed: 0'
sample_beauty2 = sample_beauty2.drop(columns=['Unnamed: 0'])

# Verificar los nombres de las columnas
print(sample_beauty2.columns)

Index(['overall', 'reviewText'], dtype='object')


In [ ]:
# REvisar nulos
sample_beauty2.isnull().sum()

overall       0
reviewText    0
dtype: int64

In [ ]:
# Revisar duplicados
sample_beauty2.duplicated().sum()

3723

In [ ]:
# Mostrar duplicados
# Mostrar todas las filas duplicadas
duplicated_rows = sample_beauty2[sample_beauty2.duplicated]
print(duplicated_rows)


       overall                                         reviewText
147          0  I was looking for a relatively inexpensive soa...
162          0                      Different color, nice product
171          0  I was looking for a relatively inexpensive soa...
186          0  This styling cream is okay but it caused my ha...
194          0  I'm a huge Essie fan and loved this color in t...
...        ...                                                ...
17120        1                           Just the color I wanted.
17121        1  I'm a fan of creamy cleansers over foaming cle...
17122        1  CHI hands down is the best leave in treatment ...
17126        1  I loved this bright shade a lot so much that I...
17128        1  I can't recall when I first found this color, ...

[3723 rows x 2 columns]


In [ ]:
# Borrar duplicados
sample_beauty2 = sample_beauty2.drop_duplicates()


Reviso el df

In [ ]:
sample_beauty2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13409 entries, 0 to 17131
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   overall     13409 non-null  int64 
 1   reviewText  13409 non-null  object
dtypes: int64(1), object(1)
memory usage: 314.3+ KB


#### Pipeline

In [ ]:
# !pip install --upgrade nltk
!pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=0449e62287376e7f884802adeee326bdf9fae54157fc1b54be2fa4163e4ac8f7
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
# Para normalizar y maniupalr cadenas de texto
import unicodedata
# procesamiento de lenguaje natural
import nltk
# Para tokenizar
from nltk import RegexpTokenizer
#PAra usar la raiz de la palabra
from nltk.stem import WordNetLemmatizer
#Convetir números a palabras
from num2words import num2words
# Para trabajar con expresiones regulares
import re
# Para considerar lista de palabras sin valor
from nltk.corpus import stopwords
# diviri el texto en tokens
from nltk.tokenize import word_tokenize
# analizar y extraer datos de archivos HTML
from bs4 import BeautifulSoup

# Descarga recursos necesarios
# Descargar palabras stopcord
nltk.download('stopwords')
# Agrupa palabras en sinonimos
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

Revisar si se hizo correctamente el normalizado

In [ ]:
def normalize_data(text, tokenizer, stopword_text, lemmatizer):

    normalize_text = list()

    # Limpiar HTML y normalizar texto unicode
    text = BeautifulSoup(text, "html5lib").get_text()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Procesar palabras en el texto
    for word in tokenizer.tokenize(text):
        # Convertir la palabra a minúsculas
        word = word.lower()

        # Ignorar stop words
        if word not in stopword_text:
            # Lematizar y limpiar la palabra
            clean_text = lemmatizer.lemmatize(word).strip()

            # Convertir números a palabras
            if clean_text.isdigit():
                clean_text = num2words(clean_text, lang='en')
            normalize_text.append(clean_text)

    # Unir palabras normalizadas
    return ' '.join(normalize_text)

# Configuración
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()

# Asegurar que las stopwords estén en minúsculas
stopword_text = set(stopword.lower() for stopword in stopwords.words('english'))

# Convertir 'reviewText' a tipo string
sample_beauty2['reviewText'] = sample_beauty2['reviewText'].astype(str)

# Aplicar la función de normalización a cada review
normalization_revs = sample_beauty2['reviewText'].apply(
    normalize_data,
    tokenizer=tokenizer,
    stopword_text=stopword_text,
    lemmatizer=lemmatizer
)

<ipython-input-13-30abc93bbecf>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html5lib").get_text()


In [ ]:
# Añadir la columna de reviews procesadas al DataFrame
sample_beauty2.loc[:, 'normalization_revs'] = normalization_revs

In [ ]:
# Mostrar los textos
sample_beauty2.head(2)

,overall,reviewText,normalization_revs
0,0,The color that comes out of this sunless tanne...,color come sunless tanner incredible familiar ...
1,0,"I love this brand, but this one didn't work as...",love brand one work expecting like perfect hai...


In [ ]:
sample_beauty2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13409 entries, 0 to 17131
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   overall             13409 non-null  int64 
 1   reviewText          13409 non-null  object
 2   normalization_revs  13409 non-null  object
dtypes: int64(1), object(2)
memory usage: 419.0+ KB


In [ ]:
# Guardar el DataFrame preprocesados en un archivo CSV
sample_beauty2.to_csv('/content/drive/MyDrive/preprocessing_data.csv', index=False)